In [1]:
import warnings
warnings.filterwarnings('ignore')

### Testing `PredictionService` API

In [2]:
from stfsclient import apis, config, util, sources

Tensorflow Library Available: True


TypeError: Couldn't build proto file into descriptor pool!
Invalid proto descriptor for file "stfsclient/google/protobuf/wrappers.proto":
  google.protobuf.DoubleValue.value: "google.protobuf.DoubleValue.value" is already defined in file "google/protobuf/wrappers.proto".
  google.protobuf.DoubleValue: "google.protobuf.DoubleValue" is already defined in file "google/protobuf/wrappers.proto".
  google.protobuf.FloatValue.value: "google.protobuf.FloatValue.value" is already defined in file "google/protobuf/wrappers.proto".
  google.protobuf.FloatValue: "google.protobuf.FloatValue" is already defined in file "google/protobuf/wrappers.proto".
  google.protobuf.Int64Value.value: "google.protobuf.Int64Value.value" is already defined in file "google/protobuf/wrappers.proto".
  google.protobuf.Int64Value: "google.protobuf.Int64Value" is already defined in file "google/protobuf/wrappers.proto".
  google.protobuf.UInt64Value.value: "google.protobuf.UInt64Value.value" is already defined in file "g

In [3]:
model_spec = ModelSpec(name='mnist', version=2, signature_name='predict_images')

In [4]:
model_spec

name: "mnist"
version {
  value: 2
}
signature_name: "predict_images"

In [5]:
import mnist_input_data

In [11]:
num_test = 2

In [12]:
test_data_set = mnist_input_data.read_data_sets('/tmp').test

Extracting /tmp/train-images-idx3-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz


In [13]:
predict_request = PredictRequest(model_spec=model_spec)
get_metadata_request = GetModelMetadataRequest(model_spec, 'signature_def')
prediction_service = PredictionService('0.0.0.0:8500')

In [14]:
predict_output_list = []
actual_output_list = []
metadata_output_list = []
for _ in range(num_test):
    image, label = test_data_set.next_batch(1)
    predict_request.inputs = {'images' : image[0].reshape(1,-1)}
    actual_output_list.append(label)
    predict_response = PredictResponse().copy(prediction_service.predict(predict_request._protobuf))
    predict_output_list.append(predict_response.parse_outputs())
    metadata_output_list.append(prediction_service.get_model_metadata(get_metadata_request._protobuf, 5)) # not parsed :(

In [15]:
import numpy as np

#### Truth vs Prediction

In [21]:
error = 0
for truth, pred in zip(actual_output_list, predict_output_list):
    if not np.array_equal(truth, np.argmax(pred['scores'], axis=-1)):
        error += 1
print('Accuracy: {:.2f}%'.format(100*(1-error/num_test)))

Accuracy: 100.00%


In [22]:
metadata_output_list

[model_spec {
   name: "mnist"
   version {
     value: 2
   }
 }
 metadata {
   key: "signature_def"
   value {
     type_url: "type.googleapis.com/tensorflow.serving.SignatureDefMap"
     value: "\n\305\001\n\017serving_default\022\261\001\n\036\n\006inputs\022\024\n\014tf_example:0\020\007\032\002\030\001\022)\n\006scores\022\037\n\010TopKV2:0\020\001\032\021\022\013\010\377\377\377\377\377\377\377\377\377\001\022\002\010\n\022G\n\007classes\022<\n%hash_table_Lookup/LookupTableFindV2:0\020\007\032\021\022\013\010\377\377\377\377\377\377\377\377\377\001\022\002\010\n\032\033tensorflow/serving/classify\n{\n\016predict_images\022i\n%\n\006images\022\033\n\003x:0\020\001\032\022\022\013\010\377\377\377\377\377\377\377\377\377\001\022\003\010\220\006\022$\n\006scores\022\032\n\003y:0\020\001\032\021\022\013\010\377\377\377\377\377\377\377\377\377\001\022\002\010\n\032\032tensorflow/serving/predict"
   }
 }, model_spec {
   name: "mnist"
   version {
     value: 2
   }
 }
 metadata {
   k

#### TODO: Parse `signature_def` :(

### Testing `ModelService` API

In [23]:
from config import *
from sources import *

In [24]:
version_policy_a = ServableVersionPolicy(specific=1)
version_policy_b = ServableVersionPolicy(_all=1)

In [25]:
model_config_a = ModelConfig(name='mnist', 
                             base_path='/models/mnist', 
                             model_platform='tensorflow', 
                             model_version_policy=version_policy_a)

In [26]:
model_config_b = ModelConfig(name='mnist', 
                             base_path='/models/mnist', 
                             model_platform='tensorflow', 
                             model_version_policy=version_policy_b)

### Running `A`, i.e. Load version #1

#### Reload

In [27]:
model_config_list = ModelConfigList(config=[model_config_a])
model_server_config = ModelServerConfig(model_config_list=model_config_list)

In [28]:
reload_config_request = ReloadConfigRequest(config=model_server_config)
model_service = ModelService('0.0.0.0:8500')
reload_config_response = ReloadConfigResponse().copy(model_service.reload_config(reload_config_request._protobuf, 5))

In [29]:
reload_config_response # if successful returns an empty status

status {
}

#### Get Status

In [30]:
model_spec = ModelSpec(name='mnist', signature_name='predict_images')

In [31]:
get_model_status_request = GetModelStatusRequest(model_spec=model_spec)

In [32]:
get_model_status_response = GetModelStatusResponse().copy(model_service.get_model_status(get_model_status_request._protobuf, 5))

In [33]:
get_model_status_response

model_version_status {
  version: 2
  state: END
  status {
  }
}
model_version_status {
  version: 1
  state: AVAILABLE
  status {
  }
}

Version 1 is running and 2 has been taken down.

### Running `B`, i.e. Load all versions

#### Reload

In [34]:
model_config_list = ModelConfigList(config=[model_config_b])
model_server_config = ModelServerConfig(model_config_list=model_config_list)

In [36]:
reload_config_request = ReloadConfigRequest(config=model_server_config)
model_service = ModelService('0.0.0.0:8500')
reload_config_response = ReloadConfigResponse().copy(model_service.reload_config(reload_config_request._protobuf, 2))

In [37]:
reload_config_response # if successful returns an empty status

status {
}

#### Get Status

In [43]:
model_spec = ModelSpec(name='mnist', signature_name='predict_images')

In [44]:
get_model_status_request = GetModelStatusRequest(model_spec=model_spec)

In [46]:
get_model_status_response = GetModelStatusResponse().copy(model_service.get_model_status(get_model_status_request._protobuf, 2))

In [47]:
get_model_status_response

model_version_status {
  version: 2
  state: AVAILABLE
  status {
  }
}
model_version_status {
  version: 1
  state: AVAILABLE
  status {
  }
}

Both the versions are running.

### Running config `C`, i.e. Unload all versions

#### Reload

In [48]:
model_server_config = ModelServerConfig()
# it's important you initialize the config as empty, otherwise you'll get a gRPC error
model_server_config.model_config_list.config = []  # sending an empty model_server_config

In [49]:
reload_config_request = ReloadConfigRequest(config=model_server_config)
model_service = ModelService('0.0.0.0:8500')
reload_config_response = ReloadConfigResponse().copy(model_service.reload_config(reload_config_request._protobuf, 2))

In [50]:
reload_config_response # if successful returns an empty status

status {
}

#### Get Status

In [51]:
model_spec = ModelSpec(name='mnist', signature_name='predict_images')

In [52]:
get_model_status_request = GetModelStatusRequest(model_spec=model_spec)

In [53]:
get_model_status_response = GetModelStatusResponse().copy(model_service.get_model_status(get_model_status_request._protobuf, 2))

In [ ]:
get_model_status_response

All versions have been taken down.